In [5]:
import pandas as pd
import geopandas as gpd
import arcpy
import os
import shutil
import math
import inspect

src_file_path = inspect.getfile(lambda: None)
THIS_FOLDER = os.path.dirname(os.path.abspath(src_file_path))

In [2]:
arcpy.ResetEnvironments()
gdb_name = "Summarize_Reporting_Units.gdb"
arcpy.env.overwriteOutput = True
path = os.path.join(THIS_FOLDER, gdb_name)
arcpy.CreateFileGDB_management("./", gdb_name)
arcpy.env.workspace = path
arcpy.env.scratchWorkspace = path

In [3]:
data = ["in_data/Pr_Lg_Fire.shp", "in_data/Pr_Fire_Ign.shp",
        "in_data/Pct_Cg_Lg_Fi.shp", "in_data/Pct_Cg_Fi_Ign.shp",
        "in_data/WUI_Intermix.shp", "in_data/WUI_Interface.shp",
        "in_data/FloraFaunaMerge.shp",
        "out_data/lu/LU_merged.shp", "out_data/ru/Reporting_Units_APN.shp",
        "out_data/building/Building_Intersect.shp",
        "in_data/Slope.shp", "in_data/Veg_Only.shp",
        "in_data/FHSZ.shp"]

for v in data:
    path = os.path.join(THIS_FOLDER, "{}".format(v))
    name = v.rsplit('/', 1)[-1].split(".")[0]
    arcpy.CopyFeatures_management(path, name)
    crs = arcpy.SpatialReference('NAD 1983 StatePlane California V FIPS 0405 (US Feet)')
    arcpy.Project_management(name, "{}_rpj".format(name), crs)

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Input Features: Dataset C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\out_data/building/Building_Intersect.shp does not exist or is not supported
Failed to execute (CopyFeatures).


In [ ]:
arcpy.analysis.SummarizeWithin("Reporting_Units_APN_rpj", "Pr_Lg_Fire_rpj", 
                               "A_RU_1", "KEEP_ALL", "gridcode Mean", 
                               "NO_SHAPE_SUM", "SQUAREFEET", None, "NO_MIN_MAJ", "NO_PERCENT", None)

In [ ]:
arcpy.analysis.SummarizeWithin("A_RU_1", "Pr_Fire_Ign_rpj", 
                               "A_RU_2", "KEEP_ALL", "gridcode Mean", 
                               "NO_SHAPE_SUM", "SQUAREFEET", None, "NO_MIN_MAJ", "NO_PERCENT", None)

In [ ]:
arcpy.analysis.SummarizeWithin("A_RU_2", "Pct_Cg_Lg_Fi_rpj", 
                               "A_RU_3", "KEEP_ALL", "gridcode Mean", 
                               "NO_SHAPE_SUM", "SQUAREFEET", None, "NO_MIN_MAJ", "NO_PERCENT", None)

In [ ]:
arcpy.analysis.SummarizeWithin("A_RU_3", "Pct_Cg_Fi_Ign_rpj", 
                               "A_RU_4", "KEEP_ALL", "gridcode Mean", 
                               "NO_SHAPE_SUM", "SQUAREFEET", None, "NO_MIN_MAJ", "NO_PERCENT", None)

In [ ]:
arcpy.analysis.SummarizeWithin("A_RU_4", "WUI_Intermix_rpj", 
                               "A_RU_5", "KEEP_ALL", None, 
                               "ADD_SHAPE_SUM", "SQUAREFEET", None, "NO_MIN_MAJ", "NO_PERCENT", None)
arcpy.analysis.SummarizeWithin("A_RU_5", "WUI_Interface_rpj", 
                               "A_RU_6", "KEEP_ALL", None, 
                               "ADD_SHAPE_SUM", "SQUAREFEET", None, "NO_MIN_MAJ", "NO_PERCENT", None)

In [ ]:
arcpy.analysis.SummarizeWithin("A_RU_6", "LU_Merged_rpj", 
                               "A_RU_7", "KEEP_ALL", "LU_Cat Max", 
                               "NO_SHAPE_SUM", "SQUAREFEET", "LU_Cat", "ADD_MIN_MAJ", "ADD_PERCENT", None)

In [ ]:
arcpy.analysis.SummarizeWithin("A_RU_7", "FloraFaunaMerge_rpj", 
                               "A_RU_8", "KEEP_ALL", "Flora_Faun Mean", 
                               "NO_SHAPE_SUM", "SQUAREFEET", None, "NO_MIN_MAJ", "NO_PERCENT", None)

In [ ]:
arcpy.analysis.SummarizeWithin("A_RU_8", "Building_Intersect_rpj", 
                               "A_RU_9", "KEEP_ALL", "", 
                               "ADD_SHAPE_SUM", "SQUAREFEET", None, "NO_MIN_MAJ", "NO_PERCENT", None)

In [ ]:
arcpy.analysis.SummarizeWithin("A_RU_9", "Slope_rpj", 
                               "A_RU_10", "KEEP_ALL", "gridcode Mean", 
                               "NO_SHAPE_SUM", "SQUAREFEET", None, "NO_MIN_MAJ", "NO_PERCENT", None)

In [ ]:
arcpy.analysis.SummarizeWithin("A_RU_10", "Veg_Only_rpj", 
                               "A_RU_11", "KEEP_ALL", "gridcode Max", 
                               "NO_SHAPE_SUM", "SQUAREFEET", "gridcode", "ADD_MIN_MAJ", "ADD_PERCENT", None)

In [ ]:
arcpy.analysis.SummarizeWithin("A_RU_11", "FHSZ_rpj", 
                               "A_RU_12", "KEEP_ALL", "HAZ_CODE Max", 
                               "NO_SHAPE_SUM", "SQUAREFEET", "HAZ_CODE", "ADD_MIN_MAJ", "ADD_PERCENT", None)

In [ ]:
fieldObs = arcpy.ListFields("A_RU_12")  

fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)
del fieldObs  
fieldCount = len(fieldNames) 

with arcpy.da.UpdateCursor("A_RU_12", fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0"  
        curU.updateRow(rowU)

del curU

In [ ]:
arcpy.AddField_management("A_RU_12", "WUI_Interm", "LONG")
arcpy.AddField_management("A_RU_12", "WUI_Interf", "LONG")

codeblock = """
def get_val(m):
    if m > 0:
        return 1
    else:
        return 0
"""

# Execute CalculateField 
arcpy.CalculateField_management("A_RU_12", "WUI_Interm", "get_val(!SUM_Area_SQUAREFEET!)", "PYTHON3", codeblock)
arcpy.CalculateField_management("A_RU_12", "WUI_Interf", "get_val(!SUM_Area_SQUAREFEET_1!)", "PYTHON3", codeblock)

arcpy.DeleteField_management("A_RU_12", 
                             ['SUM_Area_SQUAREFEET', 'SUM_Area_SQUAREFEET_1'])

In [ ]:
arcpy.AddField_management("A_RU_12", "Mj_LU_Cat", "LONG")

codeblock = """
def get_val(a, m):
    if m >= 50:
        return a
    else:
        return 0
"""

# Execute CalculateField 
arcpy.CalculateField_management("A_RU_12", "Mj_LU_Cat", "get_val(!Majority_LU_Cat!, !Majority_LU_Cat_Percent!)", "PYTHON3", codeblock)

arcpy.DeleteField_management("A_RU_12", 
                             ['MAX_LU_Cat', 'Minority_LU_Cat', 'Majority_LU_Cat', 'Majority_LU_Cat_Percent', 'Minority_LU_Cat_Percent'])

In [ ]:
arcpy.AddField_management("A_RU_12", "Mj_Haz_Cat", "LONG")

codeblock = """
def get_val(a, m):
    if m >= 50:
        return a
    else:
        return 0
"""

# Execute CalculateField 
arcpy.CalculateField_management("A_RU_12", "Mj_Haz_Cat", "get_val(!Majority_HAZ_CODE!, !Majority_HAZ_CODE_Percent!)", "PYTHON3", codeblock)

arcpy.DeleteField_management("A_RU_12", 
                             ['MAX_HAZ_CODE', 'Minority_HAZ_CODE', 'Majority_HAZ_CODE', 'Majority_HAZ_CODE_Percent', 'Minority_HAZ_CODE_Percent'])

In [ ]:
arcpy.AddField_management("A_RU_12", "Vg_Pct", "FLOAT")

codeblock = """
def get_val(a, m):
    return round(m/100, 2)
"""

# Execute CalculateField 
arcpy.CalculateField_management("A_RU_12", "Vg_Pct", "get_val(!Majority_gridcode!, !Majority_gridcode_Percent!)", "PYTHON3", codeblock)

arcpy.DeleteField_management("A_RU_12", 
                             ['MAX_gridcode', 'Minority_gridcode', 'Minority_gridcode_Percent'])

In [ ]:
arcpy.AlterField_management('A_RU_12', 
                            'MEAN_gridcode', 'Pr_Lg_Fr', 'Pr_Lg_Fr')
arcpy.AlterField_management('A_RU_12', 
                            'MEAN_gridcode_1', 'Pr_Fi_Ign', 'Pr_Fi_Ign')
arcpy.AlterField_management('A_RU_12', 
                            'MEAN_gridcode_12', 'Pc_Cg_LgFi', 'Pc_Cg_LgFi')
arcpy.AlterField_management('A_RU_12', 
                            'MEAN_gridcode_12_13', 'Pc_Fi_Ign', 'Pc_Fi_Ign')
arcpy.AlterField_management('A_RU_12', 
                            'MEAN_gridcode_12_13_14', 'Slp', 'Slp')

arcpy.AlterField_management('A_RU_12', 
                            'MEAN_Flora_Faun', 'Fl_Fa', 'Fl_Fa')

arcpy.AlterField_management('A_RU_12', 
                            'SUM_Area_SQUAREFEET_12_13', 'Buil_Ar', 'Buil_Ar')

arcpy.DeleteField_management("A_RU_12", 
                             ['SUM_Area_SQUAREFEET_12', 'Join_ID_1', 'Shape_Leng', 'SUM_Area_SQUAREFEET_12_13_14', 
                              'SUM_Area_SQUAREFEET_12_13_14_15', "Majority_gridcode", "Majority_gridcode_Percent"])

In [ ]:
arcpy.AddField_management("A_RU_12", "Buil_Rat", "FLOAT")

arcpy.CalculateField_management("A_RU_12", "Buil_Rat", 
                                '!Buil_Ar!/!Shape_Area!', "PYTHON3")

In [ ]:
fieldObs = arcpy.ListFields("A_RU_12") 

fieldNames = []  
for field in fieldObs: 
    print(field.name)
    fieldNames.append(field.name)  
del fieldObs

In [ ]:
dir_path = os.path.join(THIS_FOLDER, "out_data/eems")
shutil.rmtree(dir_path)
os.mkdir(dir_path)

arcpy.conversion.FeatureClassToShapefile("A_RU_12", 
                                        os.path.join(THIS_FOLDER, "out_data/eems"))

In [ ]:
gdf = gpd.read_file(os.path.join(THIS_FOLDER, "out_data/eems/A_RU_12.shp"))
gdf.info()

In [ ]:
gdf.to_csv("out_data/eems/A_RU_12.csv", index=False)